## 数据处理流程
1. 数据读入
2. 数据清洗
3. 数据无量纲化处理
4. 特征选择
5. 特征提取
6. 聚类

# 将数据文件处理成电脑可处理的小规模的数据

## 数据读入

In [5]:
# import libs
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans

In [6]:
# 从文件夹'newdata/'中读取数据文件
filedir = 'newdata/'
files = os.listdir(filedir)

In [7]:
nrows = 10000 # 每个文件中读取前nrows行数据
alldata = pd.DataFrame() #新建空的DataFrame
for f in files: # 循环读入数据，加入到新建的DataFrame中
    tdata = pd.read_csv(filedir + f, nrows=nrows)
    print('read ' + str(tdata.shape[0]) + ' rows from' + f)
    alldata = alldata.append(tdata, ignore_index=True)
    #alldata = alldata.append(tdata)

read 10000 rows fromSyn_data.csv
read 10000 rows fromUDP-lag_data.csv
read 7765 rows fromBENIGN_data.csv
read 10000 rows fromTFTP_data.csv
read 439 rows fromWebDDoS_data.csv


In [8]:
cols = alldata.columns
alldata = alldata.drop(cols[0:2],axis = 1) # 删除两列必然没用的数据
alldata = alldata.dropna(how='any') # 删除含有nan值的数据
alldata.reset_index(inplace=True, drop=True) # 重置index
alldata
# 36079 rows × 87 columns

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,172.16.0.5-192.168.50.1-53058-53058-6,172.16.0.5,53058.0,192.168.50.1,53058.0,6.0,2018-12-01 13:30:30.741451,115799309.0,19.0,2.0,...,646237.483665,1709809.0,1.0,1.426117e+07,3.220326e+06,21714933.0,11043464.0,0,1.0,Syn
1,172.16.0.5-192.168.50.1-32237-32237-6,172.16.0.5,32237.0,192.168.50.1,32237.0,6.0,2018-12-01 13:30:30.741452,113973933.0,16.0,0.0,...,19.595918,49.0,1.0,1.628198e+07,2.573891e+06,20019405.0,11993631.0,0,1.0,Syn
2,172.16.0.5-192.168.50.1-60495-9840-6,172.16.0.5,60495.0,192.168.50.1,9840.0,6.0,2018-12-01 13:30:30.741501,112.0,2.0,2.0,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1.0,Syn
3,172.16.0.5-192.168.50.1-59724-59724-6,172.16.0.5,59724.0,192.168.50.1,59724.0,6.0,2018-12-01 13:30:30.741563,105985004.0,16.0,0.0,...,17.705259,48.0,1.0,1.514071e+07,3.077366e+06,20954123.0,11120336.0,0,1.0,Syn
4,172.16.0.5-192.168.50.1-60496-32538-6,172.16.0.5,60496.0,192.168.50.1,32538.0,6.0,2018-12-01 13:30:30.741565,1.0,2.0,0.0,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36074,172.16.0.5-192.168.50.1-53363-80-6,172.16.0.5,53363.0,192.168.50.1,80.0,6.0,2018-12-01 13:30:25.009905,242.0,4.0,2.0,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1.0,WebDDoS
36075,172.16.0.5-192.168.50.1-59588-80-6,172.16.0.5,59588.0,192.168.50.1,80.0,6.0,2018-12-01 13:30:27.645563,127.0,1.0,1.0,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1.0,WebDDoS
36076,172.16.0.5-192.168.50.1-59588-80-6,192.168.50.1,80.0,172.16.0.5,59588.0,6.0,2018-12-01 13:30:27.645691,118.0,1.0,2.0,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0.0,WebDDoS
36077,172.16.0.5-192.168.50.1-53362-80-6,192.168.50.1,80.0,172.16.0.5,53362.0,6.0,2018-12-01 13:30:30.661269,2951.0,1.0,1.0,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0.0,WebDDoS


In [10]:
L = set(alldata[' Label'])
labels = []
for l in L:
    labels.append(l)
labels
    

['WebDDoS', 'UDP-lag', 'BENIGN', 'Syn', 'TFTP']

## 数据清洗

In [12]:
alldata.dropna(how='any',inplace=True) # 再次删除含有空值的行

# 删除非数值型数据，这些数据暂时不用于聚类处理，但不代表这些数据没用
dropset = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
           ' Fwd Header Length.1', 'SimillarHTTP', ' Inbound']
# 删除含有大量负数的列
drop_nega = [' Fwd Header Length', 'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward']
alldata.drop(dropset, axis=1,inplace=True)
alldata.drop(drop_nega, axis=1,inplace=True)
cols = alldata.columns
# len(dropset),cols.size

# 将labels替代成数值型值
for i in range(len(labels)):
    alldata.replace(to_replace =labels[i], value = i, inplace = True)
    print(labels[i] + ' was replace to ' + str(i))

KeyError: "['Flow ID' ' Source IP' ' Source Port' ' Destination IP'\n ' Destination Port' ' Protocol' ' Timestamp' ' Fwd Header Length.1'\n 'SimillarHTTP' ' Inbound'] not found in axis"

In [13]:
# 删除数据中inf的值
idx_tuple = np.where(alldata.max(axis=1).values == np.inf)
idx = list(idx_tuple[0])
alldata.drop(axis=0, index = idx, inplace = True)
alldata.reset_index(inplace=True, drop=True)
# 删除数据中含负数的数据条数
idx_tuple = np.where(alldata.min(axis=1).values < 0)
idx = list(idx_tuple[0])
alldata.drop(axis=0, index = idx, inplace = True)
alldata.reset_index(inplace=True, drop=True)

In [14]:
# 拆分数据X-Y
Y = alldata[' Label']
X = alldata.drop([' Label'], axis=1)

In [15]:
# 将pandas数据转成numpy array
X = X.values
Y = Y.values

In [17]:
Y

array([3, 3, 3, ..., 0, 0, 0])

## 数据无量纲化处理

In [18]:
from sklearn import preprocessing

In [19]:
# 采用两种不同的标准化方式
#MaxAbs标准化
#建立MinMaxScaler对象
maxabs = preprocessing.MaxAbsScaler()
# 标准化处理
data_maxabs = maxabs.fit_transform(X)

#zscore标准化
zscore = preprocessing.StandardScaler()
#zscore标准化
X_zscore = zscore.fit_transform(X)

## 特征选择

In [20]:
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from numpy import array

In [21]:
## 递归特征消除法
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
X_new = RFE(estimator=LogisticRegression(), n_features_to_select=10).fit_transform(data_maxabs, Y)

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

In [22]:
help(RFE)

Help on class RFE in module sklearn.feature_selection._rfe:

class RFE(sklearn.feature_selection._base.SelectorMixin, sklearn.base.MetaEstimatorMixin, sklearn.base.BaseEstimator)
 |  RFE(estimator, *, n_features_to_select=None, step=1, verbose=0, importance_getter='auto')
 |  
 |  Feature ranking with recursive feature elimination.
 |  
 |  Given an external estimator that assigns weights to features (e.g., the
 |  coefficients of a linear model), the goal of recursive feature elimination
 |  (RFE) is to select features by recursively considering smaller and smaller
 |  sets of features. First, the estimator is trained on the initial set of
 |  features and the importance of each feature is obtained either through
 |  any specific attribute or callable.
 |  Then, the least important features are pruned from current set of features.
 |  That procedure is recursively repeated on the pruned set until the desired
 |  number of features to select is eventually reached.
 |  
 |  Read more in

In [23]:
X_new.shape

(35978, 10)

In [24]:
X_new

array([[9.64997032e-01, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 1.00590344e-01],
       [9.49785521e-01, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 1.09245022e-01],
       [9.33336032e-07, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 7.69230769e-02, 0.00000000e+00],
       ...,
       [9.83336177e-07, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.53846154e-01, 0.00000000e+00],
       [2.45917378e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 7.69230769e-02, 0.00000000e+00],
       [7.91668956e-07, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.53846154e-01, 0.00000000e+00]])

In [25]:
data_maxabs

array([[9.64997032e-01, 4.35779817e-03, 2.44798042e-03, ...,
        6.10484160e-02, 1.97792339e-01, 1.00590344e-01],
       [9.49785521e-01, 3.66972477e-03, 0.00000000e+00, ...,
        4.87938061e-02, 1.82348476e-01, 1.09245022e-01],
       [9.33336032e-07, 4.58715596e-04, 2.44798042e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.83336177e-07, 2.29357798e-04, 2.44798042e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.45917378e-05, 2.29357798e-04, 1.22399021e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.91668956e-07, 2.29357798e-04, 2.44798042e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [26]:
# X_new = SelectKBest(lambda X, Y: array(list(map(lambda x:pearsonr(x, Y), X.o))).T, k=30).fit_transform(data_maxabs, Y)

## 特征提取

In [27]:
from sklearn.decomposition import PCA
estimator = PCA(n_components=8)
pca_X_train = estimator.fit_transform(X_new)

In [28]:
pca_X_train.shape

(35978, 8)

# 普通 Kmeans

In [29]:
help(KMeans)

Help on class KMeans in module sklearn.cluster._kmeans:

class KMeans(sklearn.base.TransformerMixin, sklearn.base.ClusterMixin, sklearn.base.BaseEstimator)
 |  KMeans(n_clusters=8, *, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='deprecated', verbose=0, random_state=None, copy_x=True, n_jobs='deprecated', algorithm='auto')
 |  
 |  K-Means clustering.
 |  
 |  Read more in the :ref:`User Guide <k_means>`.
 |  
 |  Parameters
 |  ----------
 |  
 |  n_clusters : int, default=8
 |      The number of clusters to form as well as the number of
 |      centroids to generate.
 |  
 |  init : {'k-means++', 'random'}, callable or array-like of shape             (n_clusters, n_features), default='k-means++'
 |      Method for initialization:
 |  
 |      'k-means++' : selects initial cluster centers for k-mean
 |      clustering in a smart way to speed up convergence. See section
 |      Notes in k_init for more details.
 |  
 |      'random': choose `n_clusters` o

In [30]:
kmeans = KMeans(n_clusters = 5, init='k-means++', n_init=50).fit(pca_X_train)

In [49]:
kmeans

KMeans(n_clusters=5, n_init=50)

In [50]:
set(kmeans.labels_)

{0, 1, 2, 3, 4}

In [51]:
y_ = kmeans.labels_

In [31]:
Y

array([3, 3, 3, ..., 0, 0, 0])

In [53]:
y_, Y

(array([3, 3, 0, ..., 2, 2, 2], dtype=int32), array([0, 0, 0, ..., 3, 3, 3]))

In [54]:
for i in range(len(Y)): # 指标筛选之后好多了
    print(y_[i], Y[i])

3 0
3 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
3 0
2 0
3 0
0 0
3 0
0 0
0 0
0 0
3 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
0 0
0 0
3 0
3 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
0 0
3 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0


0 0
0 0
0 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
3 0
3 0
3 0
3 0
0 0
0 0
0 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0


0 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
3 0
0 0
0 0
3 0
3 0
3 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
3 0
3 0
0 0
0 0
0 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
3 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
3 0
3 0
0 0
0 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
3 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0


0 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
0 0
3 0
3 0
3 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
3 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
3 0
3 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
3 0
3 0
3 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
3 0
3 0
0 0
0 0
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
3 0
0 0
0 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
3 0
3 0
3 0
3 0
3 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
3 0
3 0
3 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
0 0
0 0
3 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
3 0
0 0
3 0
3 0
0 0
3 0
3 0
0 0
3 0
3 0
3 0


1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1


...

2 2
2 2
2 2
0 2
4 2
2 2
4 2
4 2
2 2
2 2
2 2
2 2
2 2
4 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
1 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
0 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 2
0 2
2 2
2 2
2 2
2 2
4 2
4 2
4 2
2 2
2 2
0 2
0 2
0 2
2 2
0 2
2 2
4 2
0 2
4 2
0 2
4 2
0 2
0 2
4 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
0 2
4 2
4 2
2 2
2 2
2 2
2 2
4 2
2 2
4 2
2 2
2 2
2 2
0 2
2 2
4 2
2 2
2 2
2 2
2 2
4 2
2 2
2 2
2 2
0 2
2 2
0 2
2 2
2 2
2 2
2 2
2 2
2 2
4 2
2 2
4 2
2 2
2 2
4 2
2 2
2 2
2 2
2 2
0 2
2 2
2 2
4 2
2 2
4 2
2 2
2 2
2 2
4 2
4 2
2 2
4 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
0 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 2
2 2
2 2
0 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
4 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
0 2
2 2


0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4


0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4


0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4


0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
0 4
